# SLU06 - Dealing with Data Problems

This notebook has exercises covering the following topics:

- Tidy Data
- Data Entry Problems
- Missing Values

Import the following dependencies.

In [1]:
import os
import pandas as pd
import numpy as np
import copy
import hashlib
import json
import warnings
warnings.filterwarnings('ignore')

## Exercise 1

You were hired to do a study regarding houses and apartments renting in capitals of some european countries.

For that purpose, a dataset was provided to you. This dataset contains data reported on a web portal for the year of 2019. These data was reported by renting agencies from european capitals, and contains information about the number of contracts done on each month.

Note: On this web portal, the agent doesn't select month and capital from  a set of values, these variables are text based entries. This can lead to entry problems.

In [2]:
df_n_contracts = pd.read_csv(os.path.join('data', 'n_renting_contracts.csv'), index_col=0)
df_n_contracts.head()

MADRID    rome    Rome  Lisboa  madrid   Roma  Budapest   paris  \
dec          NaN     NaN   540.0   186.0  1235.0    NaN     994.0   417.0   
APRIL      198.0  1133.0   975.0  1192.0     NaN    NaN       NaN  1157.0   
jun         96.0     NaN  1277.0   937.0  1324.0  542.0       NaN     NaN   
november     NaN     NaN    62.0     NaN     NaN    NaN       NaN   474.0   
January      NaN   652.0     NaN     NaN   674.0    NaN     938.0   432.0   

          Lisbon  LISBON  Barcelona   Milan  BERLIN  budapest  \
dec       1420.0     NaN      401.0  1045.0  1085.0       NaN   
APRIL      261.0    34.0        NaN     NaN  1081.0     510.0   
jun        502.0     NaN     1147.0   638.0  1093.0       NaN   
november  1486.0   887.0      607.0  1201.0   470.0       NaN   
January    371.0     NaN        NaN     NaN  1233.0       NaN   

          lisbon, Portugal  paris, france  
dec                 1340.0          865.0  
APRIL                571.0            NaN  
jun                    NaN          109.0  
november             379.0         1315.0  
January                NaN          337.0

By looking to the dataset provided, you can see that it is not ready for analysis. It is not tidy and has data entry problems, as well as missing values.

### Exercise 1.1

Create a function named `transform_dataframe`. It should accept a not tidy DataFrame named `df_not_tidy` and 3 strings, `id_vars`, `var_name` and `value_name`. The variable `value_name` should have the value "frequency" as default. It should return a tidy DataFrame named `df_tidy`, where the index values and the column names correspond to the values of the columns named `id_vars` and `var_name`, respectively. It should also have a column with the name assigned to `value_name`, where each value represents the frequency of each observation. 

In [3]:
def transform_dataframe(df_not_tidy: pd.DataFrame,
                        id_vars: str,
                        var_name: str,
                        value_name="frequency")->pd.DataFrame:
    """
    Converts a messy DataFrame in a tidy one.
    
    """
    
    ###########################################
    
    #extract the column names
    val_vars = df_not_tidy.columns
    
    #reset the index and rename the new column to the value of id_vars 
    df_not_tidy_ = df_not_tidy.reset_index().rename(columns = {'index':id_vars})
    
    #using the function melt, turn df_not_tidy_ in df_tidy following the exposed requirements
    df_tidy = pd.melt(
    df_not_tidy_,
   id_vars=id_vars,
    value_vars=val_vars,
    var_name =var_name,
    value_name=value_name
)
    
    #return the DataFrame sorted by value_name, in descending order
    df_tidy = df_tidy.sort_values(value_name,ascending=False)
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return df_tidy

In [4]:
df_n_contracts_tidy = transform_dataframe(df_n_contracts, "month", "capital")
df_n_contracts_tidy.head()

month    capital  frequency
26      April       rome     1496.0
171  november     Lisbon     1486.0
248   JANUARY      Milan     1480.0
228  February  Barcelona     1457.0
223      Feb.  Barcelona     1453.0

In [5]:
assert df_n_contracts_tidy.shape[0] == 336
assert df_n_contracts_tidy.shape[1] == 3
assert list(df_n_contracts_tidy.columns.sort_values()) == ["capital", "frequency", "month"]
assert int(df_n_contracts_tidy.iloc[3].frequency) == 1457
assert df_n_contracts_tidy.iloc[325].capital == "lisbon, Portugal"
assert df_n_contracts_tidy.iloc[105].month == "Feb."

### Exercise 1.2

As said before, month and country capitals are free text input fields. For each capital, there are multiple agencies reporting their values. This results in multiple unique values that represent the same month or capital. 

#### Exercise 1.2.1

From `df_n_contracts_tidy`, create a variable named  `unique_months` that should contain all unique values from column `month`.   

In [6]:
unique_months = df_n_contracts_tidy.month.unique()
# YOUR CODE HERE

#raise NotImplementedError()
print(unique_months)

['April' 'november' 'JANUARY' 'February' 'Feb.' 'dec' 'September'
 'OCTOBER' 'June' 'October' 'DECEMBER' 'jun' 'March' 'August' 'May'
 'January' 'APRIL' 'July' 'Jul' 'JAN' 'MARCH']


In [7]:
assert len(unique_months)
assert hashlib.sha256(json.dumps(list(np.sort(unique_months))).encode()).hexdigest() == "f55c747a7f9c637d933db54143de9ac758b6ed645c602ac0f875a38b340da3f0"

#### Exercise 1.2.2

Then, create a function called `clean_month` that accepts a dataframe and cleans the values on column `month`. All values should be represented by the first 3 characters in upper case.   
Hint: Use method [slice](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html#pandas.Series.str.slice).

In [8]:
def clean_month(df: pd.DataFrame)->pd.DataFrame:
    """
    This function cleans the month column
    """
    df_cleaned = copy.copy(df)
    
    #extract the first 3 characters
    df_cleaned["month"] = df_cleaned.month.str[0:3]
    
    #convert month to upper case
    df_cleaned["month"] = df_cleaned.month.str.upper()
    
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return df_cleaned

In [9]:
df_cleaned_month = clean_month(df_n_contracts_tidy)
df_cleaned_month.head()

month    capital  frequency
26    APR       rome     1496.0
171   NOV     Lisbon     1486.0
248   JAN      Milan     1480.0
228   FEB  Barcelona     1457.0
223   FEB  Barcelona     1453.0

In [10]:
assert len(np.sort(df_cleaned_month.month.unique())) == 12
assert hashlib.sha256(json.dumps(list(np.sort(df_cleaned_month.month.unique()))).encode()).hexdigest() == "08f40eeac558bcbd1fa0fe9b19c7c74dd0515187f9123777e856072a5c08b73a"
assert df_cleaned_month.month.iloc[36] == "APR"

#### Exercise 1.2.3

Assign to a list named `lisbon_values`, all the values on the column capital that represent this city. Return the list sorted in ascending order. 

In [16]:
lisbon_values= list(['Lisbon','LISBON','lisbon, Portugal','Lisboa'])
lisbon_values.sort()
# YOUR CODE HERE
#raise NotImplementedError()

In [17]:
assert len(lisbon_values) == 4
assert lisbon_values[0] == "LISBON"
assert hashlib.sha256(json.dumps(lisbon_values).encode()).hexdigest() == "aafabf9270c0429775eb3bf7c1d9b6b7ea9139f8a77392b286d0f8fbd56826c1"

In [18]:
df_cleaned_month.head()

month    capital  frequency
26    APR       rome     1496.0
171   NOV     Lisbon     1486.0
248   JAN      Milan     1480.0
228   FEB  Barcelona     1457.0
223   FEB  Barcelona     1453.0

#### Exercise 1.2.4

Now is time to clean capitals. They should be with first character in upper case and the remainings in lower case. Also, we need to handle the following problems.

- Capitals should be written in English;
- Rows with cities that don't represent capitals should be removed from the dataframe;
- References to countries should be removed;

Create a function named `clean_capitals` that accepts a dataframe and cleans the column `capital`.

In [52]:
def clean_capitals(df: pd.DataFrame)->pd.DataFrame:
    """
    This function cleans the capital column
    """
    df_cleaned = copy.copy(df)
    
    #remove country reference
    df_cleaned["capital"] = df_cleaned.capital.replace({'lisbon, Portugal':'lisbon','paris, france':'paris'})              
    
    #clean formating capital, CAPITAL -> Capital
    df_cleaned["capital"] = df_cleaned.capital.replace({'rome':'Rome', 'MADRID':'Madrid', 'madrid':'Madrid','LISBON':'Lisbon', 'lisbon':'Lisbon', 'paris':'Paris',
       'BERLIN':'Berlin', 'budapest':'Budapest','Lisboa':'Lisboa'})
    
    #replace cities that are not capitals by "invalid"
    df_cleaned["capital"] = df_cleaned.capital.replace({'Milan':'invalid','Barcelona':'invalid'})
    #write all the capitals in english
    df_cleaned["capital"] = df_cleaned.capital.replace({'Roma':'Rome','Lisboa':'Lisbon'}) 
    #df_cleaned["capital"] = df_cleaned.capital.duplicated(keep='first')
    #df_cleaned = df_cleaned.capitaldrop_duplicates()
    #df_cleaned.capital = df_cleaned.capital.str.strip()
    #df_cleaned["capital"] = df_cleaned.capital.replace('invalid',np.nan)
    df_cleaned["capital"] = df_cleaned.capital.dropna(axis=0)
    #df_cleaned["capital"] = df_cleaned.capital.drop((df_cleaned.capital=='invalid').index,axis=0)
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return df_cleaned

In [53]:
df_cleaned_month_capital = clean_capitals(df_cleaned_month)
df_cleaned_month_capital.capital.unique()

array(['Rome', 'Lisbon', 'invalid', 'Madrid', 'Budapest', 'Paris',
       'Berlin'], dtype=object)

In [26]:
df_cleaned_month_capital.capital.unique()

array([False, nan, True], dtype=object)

In [ ]:
assert len(np.sort(df_cleaned_month_capital.capital.unique())) == 6
assert hashlib.sha256(json.dumps(list(np.sort(df_cleaned_month_capital.capital.unique()))).encode()).hexdigest()=="48660e0bc5eead63cba5e5ea19b31cf992bb63b5f027f7e7a4fe33793029640d"
assert df_cleaned_month_capital.capital.iloc[50] == 'Paris'

### Exercise 1.3

#### Exercise 1.3.1

Calculate the number of missing values on column `frequency` from `df_cleaned_month_capital`, and assign it to variable `n_missing_frequency`.

In [ ]:
n_missing_frequency = df_cleaned_month_capital.frequency.isnull().sum()
# YOUR CODE HERE
#raise NotImplementedError()
print(n_missing_frequency)

In [ ]:
assert hashlib.sha256(json.dumps(int(n_missing_frequency)).encode()).hexdigest() == "b1556dea32e9d0cdbfed038fd7787275775ea40939c146a64e205bcb349ad02f"

#### Exercise 1.3.2

In this context, missing values on `frequency` means that there are no contracts done.
Replace missing values on the column `frequency`, from `df_cleaned_month_capital` DataFrame, with the value you think is more accurate.

In [ ]:
#df_cleaned_month_capital["frequency"] = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert np.nan not  in df_cleaned_month_capital.frequency
assert hashlib.sha256(json.dumps(int(df_cleaned_month_capital.frequency.iloc[135])).encode()).hexdigest()=="6d976934be74941fba578b143ba964eded443d10384e3f3d62a1ba7b4d339df8"

The dataset regarding the number of contracts done in european capitals is now ready for the study!

In [ ]:
df_cleaned_month_capital.head()

## Exercise 2

For this research, there is also another dataset, `df_contracts_agency`, that has information regarding each contract done and the agency responsible for it.

In [ ]:
df_contracts_agency = pd.read_csv(os.path.join('data', 'contracts_agency.csv'), index_col=0)
df_contracts_agency.head()

### Exercise 2.1

#### Exercise 2.1.1

Start by cleaning the column names on `df_contracts_agency`.    
Create a function named `clean_column_names`. It must accept a DataFrame and clean the column names.   

Hint: Use [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) method.

In [ ]:
def clean_column_names(df: pd.DataFrame)->pd.DataFrame:
    """
    cleans the column names of a DataFrame
    
    """
    
    df_cleaned = copy.copy(df)
    
    #dataframe with column names cleaned
    df_cleaned = df.rename(columns={"loc^ation": "location", "rat^e": "rate", "ho~use_type":"house_type"})
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return df_cleaned

In [ ]:
df_contracts_agency_col_cleaned = clean_column_names(df_contracts_agency)
df_contracts_agency_col_cleaned.columns

In [ ]:
assert hashlib.sha256(json.dumps(list(df_contracts_agency_col_cleaned.columns.sort_values())).encode()).hexdigest() == "87c3c0eff5259a91020d64ab00e1c829101115954c032e4c4e8bead101782770"

#### Exercise 2.1.2

Create a function named `split_location` that should split the column `location` in two distinct columns, `city` and `country`. After creating these columns, drop the column `location`. 

In [ ]:
def split_location(df: pd.DataFrame)->pd.DataFrame:
    """
    Creation of city and country columns from location column
    
    """
    
    df_cleaned = copy.copy(df)
    
    #DataFrame with column city and country
    #city_country = df_cleaned[['city','country']]
    
    #add city and country to df_cleaned
    city_country  = df_cleaned.location.str.split(pat=',', expand=True)
    city_country.columns = ['city', 'country']
    df_cleaned = df_cleaned.drop('location',axis = 1)
    pd.concat([df_cleaned,city_country],axis =1)
    
    #drop location
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return pd.concat([city_country,df_cleaned],axis = 1)

In [ ]:
df_contracts_agency_split_loc = split_location(df_contracts_agency_col_cleaned)
df_contracts_agency_split_loc.head()

In [ ]:
assert df_contracts_agency_split_loc.shape[1] == 9
assert hashlib.sha256(json.dumps(list(df_contracts_agency_split_loc.columns.sort_values())).encode()).hexdigest() == "205332ef1f32072786e8c17b447ee5afda1ab18f833d47afc7c752a63e914d34"
assert "Paris" in df_contracts_agency_split_loc.city.values
assert "France" in df_contracts_agency_split_loc.country.values

#### Exercise 2.1.3

From DataFrame `df_contracts_agency_split_loc`, create two DataFrames, `df_agencies` and `df_contract` with the information regarding agencies and contracts, respectively.

Note: Because each agency only acts on one `location`, this information is relevant for both agencies and contracts. Also, `agency_id` acts as a bridge between both DataFrames.

In [ ]:
#df_agencies = ...
#df_contracts = ...

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
df_agencies.head()

In [ ]:
df_contracts.head()

In [ ]:
assert df_agencies.shape[1]==4
assert hashlib.sha256(json.dumps(list(df_agencies.columns.sort_values())).encode()).hexdigest() == "2b20996c7aa3736584469b70378c0992d0fed7cdf36eb20ed205d01f1da09fc2"
assert df_contracts.shape[1]==8
assert hashlib.sha256(json.dumps(list(df_contracts.columns.sort_values())).encode()).hexdigest() == "5218fd4b0183d357cc438c32fe97119521e4f4c863a89848cec275098a04c170"

### Exercise 2.2

Now let's clean the datasets `df_agencies`and `df_contracts`.   

#### Exercise 2.2.1

Let's start by cleaning `df_agencies` following the instructions below. 

#### Exercise 2.2.1.1

One agency can only have one rate. Create a sorted (ascending) list named `agencies_with_multiple_rates` with unique agencies id's that have more than one rate assigned.

In [ ]:
#agencies_with_multiple_rates = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert isinstance(agencies_with_multiple_rates, list)
assert len(agencies_with_multiple_rates) == 7, "did you check for duplicates?"
assert hashlib.sha256(json.dumps(agencies_with_multiple_rates).encode()).hexdigest() == "fe0b48f998e1cc9cb96dddce1aaf4607765824f0ff98a69d86b6d1341970e82a"

#### Exercise 2.2.1.2

Create a dataframe named `df_agencies_cleaned` from the dataframe `df_agencies`, and replace the rates for `agencies_with_multiple_rates` by the mean.   

Hint: Use the function [isin](https://numpy.org/devdocs/reference/generated/numpy.isin.html) to filter the agencies that have multiple rates.

In [ ]:
# df_agencies_cleaned = ...

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
df_agencies_cleaned.head()

In [ ]:
assert int(df_agencies_cleaned.rate.sum()) == 1336

#### Exercise 2.2.1.3

Drop the duplicated rows on `df_agencies_cleaned` dataframe and assign it to a new dataframe called `df_agencies_no_duplicates`. 

In [ ]:
#df_agencies_no_duplicates = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert df_agencies_no_duplicates.shape[0] == 25

The dataset with information regarding the agencies of european capitals is now ready for the study!

In [ ]:
df_agencies_no_duplicates.head()

#### Exercise 2.2.2

Clean `df_contracts` following the same instructions.   
Hint: Look closely to unique values of the features that are not id's. 

Problems that you need to handle:    
- Inconsistent values should be handle as missing values;   

After dealing with these entry problems:
- Drop duplicated observations;   
- Missing values for categorical features should be replaced by `unknown`;
- Missing values for numerical features should be replaced by the mean;

Create a function named `clean_contracts` that applies these operations on a dataframe.   

Hint: Check the distribution of values for the column rent. Are there values too low or too high? If yes, does it look like data entry problems?   
Hint: For categorical values, check if there are strings that are actually representing missing values.

In [ ]:
df_contracts.head()

In [ ]:
df_contracts.rent.unique()

In [ ]:
def clean_contracts(df: pd.DataFrame)->pd.DataFrame:
    
    df_cleaned = copy.copy(df)
   
    #df_cleaned= ...
    
    # YOUR CODE HERE
    raise NotImplementedError()
    return df_cleaned

In [ ]:
df_contracts_cleaned = clean_contracts(df_contracts)
df_contracts_cleaned.head()

In [ ]:
house_types = sorted(df_contracts_cleaned.house_type.unique())
house_types.sort()
assert len(house_types) == 3
assert hashlib.sha256(json.dumps(house_types).encode()).hexdigest() == "2f9da7a5b1ba79b47b611b9af1fc779cd0c22547f02778d4984bd190b70b5d05"
assert df_contracts_cleaned.shape == (500, 8)
assert int(df_contracts_cleaned.rent.sum())==321228

The dataset with information regarding the agencies of european capitals is now ready for the study!

In [ ]:
df_contracts_cleaned.head()